# Metadata GRPC API example

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress.

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


# Example description

Simple, two-node graph that shows how Metadata is handled in GRPC API mode.

Model used in this example is defined in [graph_metadata](https://docs.seldon.io/projects/seldon-core/en/latest/examples/graph-metadata.html) notebook.


In [3]:
%%writefile example-grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-grpc
spec:
  name: test-deployment
  transport: grpc
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.4
          name: node-one
          env:           
          - name: MODEL_METADATA
            value: |
              ---
              name: node-one
              versions: [ generic-node/v0.4 ]
              platform: seldon
              inputs:
              - messagetype: tensor
                schema:
                  names: [one-input]
              outputs:
              - messagetype: tensor
                schema:
                  names: [one-output]
              custom:
                author: seldon-dev
        - image: seldonio/metadata-generic-node:0.4
          name: node-two
          env:
         
          - name: MODEL_METADATA
            value: |
              ---
              name: node-two
              versions: [ generic-node/v0.4 ]
              platform: seldon
              inputs:
              - messagetype: tensor
                schema:
                  names: [two-input]
              outputs:
              - messagetype: tensor
                schema:
                  names: [two-output]
              custom:
                author: seldon-dev
    graph:
      name: node-one
      type: MODEL
      children:
      - name: node-two
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting example-grpc.yaml


In [4]:
!kubectl apply -f  example-grpc.yaml

seldondeployment.machinelearning.seldon.io/graph-metadata-grpc created


In [5]:
%%bash
kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=graph-metadata-grpc -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "graph-metadata-grpc-example-0-node-one-node-two" rollout to finish: 0 of 1 updated replicas are available...
deployment "graph-metadata-grpc-example-0-node-one-node-two" successfully rolled out


In [6]:
%%bash
cd ../../../executor/proto && grpcurl \
    -d '{"name": "node-one"}' \
    -rpc-header seldon:graph-metadata-grpc -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/ModelMetadata

{
  "name": "node-one",
  "versions": [
    "generic-node/v0.4"
  ],
  "platform": "seldon",
  "inputs": [
    {
      "messagetype": "tensor",
      "schema": {
            "names": [
                  "one-input"
                ]
          }
    }
  ],
  "outputs": [
    {
      "messagetype": "tensor",
      "schema": {
            "names": [
                  "one-output"
                ]
          }
    }
  ],
  "custom": {
    "author": "seldon-dev"
  }
}


In [7]:
%%bash
cd ../../../executor/proto && grpcurl \
    -d '{"name": "node-two"}' \
    -rpc-header seldon:graph-metadata-grpc -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/ModelMetadata

{
  "name": "node-one",
  "versions": [
    "generic-node/v0.4"
  ],
  "platform": "seldon",
  "inputs": [
    {
      "messagetype": "tensor",
      "schema": {
            "names": [
                  "one-input"
                ]
          }
    }
  ],
  "outputs": [
    {
      "messagetype": "tensor",
      "schema": {
            "names": [
                  "one-output"
                ]
          }
    }
  ],
  "custom": {
    "author": "seldon-dev"
  }
}


In [8]:
%%bash
cd ../../../executor/proto && grpcurl \
    -rpc-header seldon:graph-metadata-grpc -rpc-header namespace:seldon \
    -plaintext -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/GraphMetadata

{
  "name": "example",
  "models": {
    "node-one": {
      "name": "node-one",
      "versions": [
        "generic-node/v0.4"
      ],
      "platform": "seldon",
      "inputs": [
        {
          "messagetype": "tensor",
          "schema": {
                "names": [
                      "one-input"
                    ]
              }
        }
      ],
      "outputs": [
        {
          "messagetype": "tensor",
          "schema": {
                "names": [
                      "one-output"
                    ]
              }
        }
      ],
      "custom": {
        "author": "seldon-dev"
      }
    },
    "node-two": {
      "name": "node-two",
      "versions": [
        "generic-node/v0.4"
      ],
      "platform": "seldon",
      "inputs": [
        {
          "messagetype": "tensor",
          "schema": {
                "names": [
                      "two-input"
                    ]
              }
        }
      ],
      "outputs": [
        {
 

In [9]:
!kubectl delete -f example-grpc.yaml

seldondeployment.machinelearning.seldon.io "graph-metadata-grpc" deleted
